# 讀 Markdown 轉成 Pandas DataFrame

In [25]:
import re
import pandas as pd

def parse_markdown(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    data = [{'text': line.strip(), 'bullet': None, 'tags': [], 'indent_level': len(line) - len(line.lstrip()), 'id': i, 'list_parent': i} for i, line in enumerate(lines)]
    df = pd.DataFrame(data)

    df = extract_bullets(df)
    return df

def extract_bullets(df):
    for i, row in df.iterrows():
        line = row['text']
        bullet_match = re.match(r'^\s*[-*1-9]+\.?\s+', line)
        if bullet_match:
            bullet = bullet_match.group().strip()
            df.at[i, 'bullet'] = bullet
            df.at[i, 'text'] = line[len(bullet):].strip()
    return df

In [26]:
df = parse_markdown('test_md/test.md')

In [27]:
df

,text,bullet,tags,indent_level,id,list_parent
0,的味道去哪裡了的要也不知 #h1,-,[],0,0,0
1,然就，冰淇淋的衣：露出想要的慾望好可，清的的很的前種程度，居居然有給了，不一跟我即果。次有重...,-,[],4,1,1
2,體了不下班，得好也沒的原受到之前。 #h2,-,[],4,2,2
3,的點個人，他真的可以已經至，根好像默默片是拿憶⋯以在一起了就終前道真的很電⋯我陣子應該就知道...,-,[],8,3,3
4,下這全世界⋯謝謝是因為很適合 #h2,-,[],4,4,4
5,加出現盼開了悲傷角結弦： #ol,-,[],8,5,5
6,的是的好，黏土的地方,1.,[],12,6,6
7,一樣是看起來大家一造的他。,2.,[],12,7,7
8,什麼時大家是會做，,3.,[],12,8,8
9,下午判斷標題，樣不給道他們有對都沒有就是，製造偷偷什麼不讓我亮把自真的很，不在意可以啊個人負...,-,[],8,9,9


In [12]:
# df_list_starts = df.loc[df['tags'].str.contains('#ul|#ol|#listToTable')]
df_list_starts = df[df['tags'].apply(lambda x: any(tag in ['ol', 'ul', 'listToTable'] for tag in x))].index
df_list_starts

Index([], dtype='int64')

In [6]:
for list_start in list_starts:
    list_type = next(tag for tag in df.at[list_start, 'tags'] if tag in ['ol', 'ul', 'listToTable'])
    indent_level = df.at[list_start, 'indent_level']

    list_info[list_id] = {
        'type': list_type,
        'indent': indent_level,
        'start': list_start
    }

    # 向下遍历找到列表结束行
    for i in range(list_start, len(df)):
        if df.at[i, 'indent_level'] <= indent_level:
            list_info[list_id]['end'] = i - 1
            break
        df.at[i, 'tags'].append('#listItem')
        df.at[i, 'list_parent'] = list_id
    
    list_id += 1

# 填充 list_parent 列
df['list_parent'] = df['list_parent'].fillna(-1).astype(int)

return df, list_info

In [4]:
df = locate_lists(df)

In [5]:
df

,text,bullet,tags,indent_level,id
0,的味道去哪裡了的要也不知,-,[#h1],0,0.0
1,然就，冰淇淋的衣：露出想要的慾望好可，清的的很的前種程度，居居然有給了，不一跟我即果。次有重...,-,[],4,1.0
2,體了不下班，得好也沒的原受到之前。,-,[#h2],4,2.0
3,的點個人，他真的可以已經至，根好像默默片是拿憶⋯以在一起了就終前道真的很電⋯我陣子應該就知道...,-,[],8,3.0
4,下這全世界⋯謝謝是因為很適合,-,[#h2],4,4.0
5,加出現盼開了悲傷角結弦：,-,[#ol],8,5.0
6,的是的好，黏土的地方,1.,[],12,6.0
7,一樣是看起來大家一造的他。,2.,[],12,7.0
8,什麼時大家是會做，,3.,[],12,8.0
9,下午判斷標題，樣不給道他們有對都沒有就是，製造偷偷什麼不讓我亮把自真的很，不在意可以啊個人負...,-,[],8,9.0


# 將原始文字轉換成內文

找到 list，包括 #ul, #ol, #listToTable，保留 indents/ bullets